## Import Libraries

In [1]:
import GetTesouroDiretoData as gtd
import pandas as pd

## Download Tesouro Direto Dataset

In [ ]:
gtd.TesouroDiretoDatasets().download_all_data('TesouroDireto')

## Data Analysis

In [2]:
operacoes = pd.read_parquet('TesouroDireto_operacoes.parquet')

In [3]:
investidores = pd.read_parquet('TesouroDireto_investidores.parquet')